<a href="https://colab.research.google.com/github/Nastiiasaenko/ECE661-HW2/blob/main/simplenn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training SimpleNN on CIFAR-10
In this project, you will use the SimpleNN model to perform image classification on CIFAR-10. CIFAR-10 orginally contains 60K images from 10 categories. We split it into 45K/5K/10K images to serve as train/valiation/test set. We only release the ground-truth labels of training/validation dataset to you.

## Step 0: Set up the SimpleNN model
As you have practiced to implement simple neural networks in Homework 1, we just prepare the implementation for you.

In [ ]:
%cd ..

/content


In [ ]:
## Some additional libraries used in this notebook
!pip install tqdm
!pip install torchsummary


In [ ]:
import tqdm

In [ ]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# define the SimpleNN mode;
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1   = nn.Linear(16*6*6, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

### Question (a)
Here is a sanity check to verify the implementation of SimpleNN.
You need to:
1. Write down your code.
2. **In the PDF report**, give a brief description on how the code helps you know that SimpleNN is implemented correctly.

In [ ]:
#############################################
# your code here
# sanity check for the correctness of SimpleNN
nn_model = SimpleNN()
sample_output = torch.randn(1, 3, 32, 32) # Batch size of 1, with CIFAR-10 image dimensions
output = nn_model(sample_output)
print(output.size())


#############################################

torch.Size([1, 10])


**Question (a) answer**: The expected output shape should be (1, 10), as there are 10 classes in the CIFAR-10 dataset.

##### Step 2.
Count the total number
of parameters of all conv/FC layers and see if it meets your expectation.

In [ ]:
total_params = 0
for name, param in nn_model.named_parameters():
    if param.requires_grad:
        layer_params = param.numel()
        print(f"Layer {name} has {layer_params} parameters")
        total_params += layer_params

print(f"Total number of parameters: {total_params}")



Layer conv1.weight has 600 parameters
Layer conv1.bias has 8 parameters
Layer conv2.weight has 1152 parameters
Layer conv2.bias has 16 parameters
Layer fc1.weight has 69120 parameters
Layer fc1.bias has 120 parameters
Layer fc2.weight has 10080 parameters
Layer fc2.bias has 84 parameters
Layer fc3.weight has 840 parameters
Layer fc3.bias has 10 parameters
Total number of parameters: 82030


## Step 1: Set up preprocessing functions
Preprocessing is very important as discussed in the lecture.
You will need to write preprocessing functions with the help of *torchvision.transforms* in this step.
You can find helpful tutorial/API at [here](https://pytorch.org/vision/stable/transforms.html).

### Question (b)
For the question, you need to:
1. Complete the preprocessing code below.
2. **In the PDF report**, briefly describe what preprocessing operations you used and what are the purposes of them.

Hint:
1. Only two operations are necessary to complete the basic preprocessing here.
2. The raw input read from the dataset will be PIL images.
3. Data augmentation operations are not mendatory, but feel free to incorporate them if you want.
4. Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [ ]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
#The mean and standard deviation values for the CIFAR-10 dataset—(0.4914, 0.4822, 0.4465)
# for the mean and (0.2023, 0.1994, 0.2010) for the standard deviation
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)
transform_train =  transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_val =  transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [ ]:
# do NOT change these
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT,
    mode='train',
    download=True,
    transform= transform_train    # your code
)
val_set = CIFAR10(
    root=DATA_ROOT,
    mode='val',
    download=True,
    transform= transform_val  # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set,
    batch_size = TRAIN_BATCH_SIZE,  # your code
    shuffle=True,     # your code
    num_workers=4
)
val_loader = DataLoader(
    val_set,
    batch_size=VAL_BATCH_SIZE,  # your code
    shuffle=False,     # your code
    num_workers=4
)
#############################################

141746176it [00:03, 39684539.89it/s]                               


Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

In [ ]:
# specify the device for computation
#############################################
!nvidia-smi


Mon Oct  7 00:02:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              51W / 400W |    569MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
def check_gpu_availability():
    import torch
    if torch.cuda.is_available():
        total_gpus = torch.cuda.device_count()
        print(f"GPU(s) found: {total_gpus} device(s) detected.")

        for idx in range(total_gpus):
            gpu_name = torch.cuda.get_device_name(idx)
            print(f"GPU {idx}: {gpu_name}")
    else:
        print("No GPU found. CUDA is not available.")

check_gpu_availability()



GPU(s) found: 1 device(s) detected.
GPU 0: NVIDIA A100-SXM4-40GB


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN().to(device)
print(f"Model deployed on: {device}")


Model deployed on: cuda


## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum.

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [ ]:
import torch.nn as nn
import torch.optim as optim

model = SimpleNN().to(device)

# hyperparameters, do NOT change right now
# initial learning rate

INITIAL_LR = 0.01

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4

#############################################
# your code here
# create loss function
criterion = nn.CrossEntropyLoss()

# Add optimizer
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
#############################################

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps:

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


In [ ]:
def move_to_device(*args, device):
    return [x.to(device) for x in args]

In [ ]:
# some hyperparameters
# total number of training epochs
EPOCHS = 30

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

# start the training/validation process
# the process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
best_val_acc = 0
current_learning_rate = INITIAL_LR

print("==> Training starts!")
print("="*50)
for i in range(0, EPOCHS):
    #######################
    # your code here
    # switch to train mode
    model.train()

    #######################

    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss

    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # your code here
        # copy inputs to device
        inputs, targets = move_to_device(inputs, targets, device=device)


        # compute the output and loss
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # zero the gradient
        optimizer.zero_grad()


        # backpropagation
        loss.backward()

        # apply gradient and update the weights
        optimizer.step()

        # Accumulate training loss
        train_loss += loss.item()

        # count the number of correctly predicted samples in the current batch
        predicted = torch.argmax(outputs, dim=1)
        correct_examples += (predicted == targets).sum().item()
        total_examples += targets.size(0)

        ####################################

    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # Validate on the validation dataset
    #######################
    # your code here
    # switch to eval mode
    model.eval()

    total_examples = 0
    correct_examples = 0
    val_loss = 0  # track validation loss


    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0

    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs, targets = move_to_device(inputs, targets, device=device)


            # compute the output and loss
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            _, predicted = torch.max(outputs, 1)

            # Accumulate validation loss
            val_loss += loss.item()

            # count the number of correctly predicted samples in the current batch
            predicted = torch.argmax(outputs, dim=1)
            correct_examples += (predicted == targets).sum().item()
            total_examples += targets.size(0)
            ####################################

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))

    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
           os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': model.state_dict(),
                'epoch': i,
                'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))

    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

==> Training starts!
Epoch 0:
Training loss: 1.8613, Training accuracy: 0.3187
Validation loss: 1.5311, Validation accuracy: 0.4484
Saving ...

Epoch 1:
Training loss: 1.4136, Training accuracy: 0.4922
Validation loss: 1.3416, Validation accuracy: 0.5184
Saving ...

Epoch 2:
Training loss: 1.2428, Training accuracy: 0.5586
Validation loss: 1.2146, Validation accuracy: 0.5704
Saving ...

Epoch 3:
Training loss: 1.1256, Training accuracy: 0.6037
Validation loss: 1.1471, Validation accuracy: 0.5980
Saving ...

Epoch 4:
Training loss: 1.0334, Training accuracy: 0.6366
Validation loss: 1.1362, Validation accuracy: 0.6004
Saving ...

Epoch 5:
Training loss: 0.9671, Training accuracy: 0.6616
Validation loss: 1.0773, Validation accuracy: 0.6170
Saving ...

Epoch 6:
Training loss: 0.9022, Training accuracy: 0.6801
Validation loss: 1.0477, Validation accuracy: 0.6364
Saving ...

Epoch 7:
Training loss: 0.8507, Training accuracy: 0.7017
Validation loss: 1.0112, Validation accuracy: 0.6458
Saving 

# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```

### Incorporating LR decay into my code

In [ ]:
# Hyperparameters for learning rate decay
DECAY_EPOCHS = 5  # Decay the learning rate every 10 epochs
DECAY = 0.1  # Multiply the learning rate by this factor when decaying

# Start the training process
print("==> Training starts!")
print("=" * 50)

for i in range(EPOCHS):
    #######################
    # Switch to train mode
    #######################
    model.train()

    print(f"Epoch {i+1}:")
    total_examples = 0
    correct_examples = 0
    train_loss = 0

    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Copy inputs and targets to device
        inputs, targets = move_to_device(inputs, targets, device=device)

        # Compute output and loss
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Zero the gradient
        optimizer.zero_grad()

        # Backpropagation
        loss.backward()

        # Apply gradient and update the weights
        optimizer.step()

        # Accumulate training loss
        train_loss += loss.item()

        # Count the number of correctly predicted samples
        predicted = torch.argmax(outputs, dim=1)
        correct_examples += (predicted == targets).sum().item()
        total_examples += targets.size(0)

    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print(f"Training loss: {avg_loss:.4f}, Training accuracy: {avg_acc:.4f}")

    #######################
    # Validate on validation dataset
    #######################
    model.eval()
    total_examples = 0
    correct_examples = 0
    val_loss = 0

    # Disable gradient during validation
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # Copy inputs and targets to device
            inputs, targets = move_to_device(inputs, targets, device=device)

            # Compute output and loss
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Accumulate validation loss
            val_loss += loss.item()

            # Count the number of correctly predicted samples
            predicted = torch.argmax(outputs, dim=1)
            correct_examples += (predicted == targets).sum().item()
            total_examples += targets.size(0)

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print(f"Validation loss: {avg_loss:.4f}, Validation accuracy: {avg_acc:.4f}")

    #######################
    # Save model checkpoint if validation accuracy improves
    #######################
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
            os.makedirs(CHECKPOINT_FOLDER)

        # Save the model with a dynamic filename (includes epoch and val accuracy)
        checkpoint_filename = f"simplenn_epoch_{i+1}_valacc_{avg_acc:.4f}.pth"
        print(f"Saving model to {checkpoint_filename}...")

        state = {
            'state_dict': model.state_dict(),
            'epoch': i,
            'lr': current_learning_rate
        }
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, checkpoint_filename))

    #######################
    # Learning rate decay
    #######################
    if (i + 1) % DECAY_EPOCHS == 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print(f"Learning rate decayed to {current_learning_rate:.6f}")

    print('')

print("=" * 50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")


==> Training starts!
Epoch 1:
Training loss: 0.2723, Training accuracy: 0.9040
Validation loss: 1.8949, Validation accuracy: 0.6270

Epoch 2:
Training loss: 0.2753, Training accuracy: 0.9002
Validation loss: 2.0680, Validation accuracy: 0.6154

Epoch 3:
Training loss: 0.2637, Training accuracy: 0.9046
Validation loss: 2.1572, Validation accuracy: 0.6022

Epoch 4:
Training loss: 0.2685, Training accuracy: 0.9035
Validation loss: 2.1406, Validation accuracy: 0.6266

Epoch 5:
Training loss: 0.2519, Training accuracy: 0.9108
Validation loss: 2.1623, Validation accuracy: 0.6246
Learning rate decayed to 0.001000

Epoch 6:
Training loss: 0.0981, Training accuracy: 0.9692
Validation loss: 2.2051, Validation accuracy: 0.6434

Epoch 7:
Training loss: 0.0613, Training accuracy: 0.9840
Validation loss: 2.2882, Validation accuracy: 0.6478

Epoch 8:
Training loss: 0.0489, Training accuracy: 0.9892
Validation loss: 2.3780, Validation accuracy: 0.6452

Epoch 9:
Training loss: 0.0416, Training accuracy